# 2. 데이터 탐색과 전처리

이전 장에서는 Question Generation Task의 개념과 모델, 평가지표 등에 대해 알아보았습니다. 이번 장에서는 실습에 사용할 데이터셋을 확인해보도록 하겠습니다. 

NLP 튜토리얼에서는 에세이 데이터를 활용해 학습자들을 위한 영어 문제를 생성하는 모델을 만들어 볼 것입니다. 구체적으로 3개의 문제를 만들어 볼 것입니다. 왜 에세이냐? -> 개인의 생각이 들어간 수필인데 이것을 쓰면 이러이러한 이유에서 좋다  

문제를 모두 4지선다 문제다. 
왜 4지선다 문제냐? 평가 쉽고, blah blah blah

지문 내용 기반 참 거짓 문제, WH-5 문제 생성할거다. 

참 거짓 문제란 ~~

WH-5 문제란 ~~ 

grammar 문제는 실제 GEC task에 사용되는 데이터셋에서 틀린 grammar들 위주로 구성했으므로, 실제 ESL 학생드리 헷갈려하는 문제 위주로 grammar 문제를 구성해서 유효성을 확보함. 

2.1절에서는 데이터를 다운로드 받는 방법을 살펴보겠습니다. 2.2절에서는 데이터를 시각화 해볼 것이며, 2.3절에서는 matplotlib.pyplot 에서 제공하는 subplots 함수를 활용해 여러 이미지를 하나의 출력 창에 시각화 하는 방법을 알아보겠습니다. 마지막으로 2.4절에서는 이미지 전처리를 통해 픽셀값에 정규화(normalization)을 진행해보겠습니다.



In [ ]:
# 데이터 다운로드
!wget -O 'wi+Blocness_v2.1.bea19.tar.gz'  https://github.com/SDSTony/Tutorial-Book/blob/master/book/chapters/NLP/data/wi%2Blocness_v2.1.bea19.tar.gz?raw=true
!wget -O 'release2.3.1.tar.gz' https://github.com/SDSTony/Tutorial-Book/blob/master/book/chapters/NLP/data/release2.3.1.tar.gz?raw=true

In [2]:
!wget -O 'conll2014.tar.gz' https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz

--2021-04-01 03:13:26--  https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz
Resolving www.comp.nus.edu.sg (www.comp.nus.edu.sg)... 45.60.31.225
Connecting to www.comp.nus.edu.sg (www.comp.nus.edu.sg)|45.60.31.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 643482 (628K) [application/x-gzip]
Saving to: ‘conll2014.tar.gz’

conll2014.tar.gz    100%[===================>] 628.40K   337KB/s    in 1.9s    

2021-04-01 03:13:29 (337 KB/s) - ‘conll2014.tar.gz’ saved [643482/643482]



- output file 이름 정의할 때: `-O`

In [ ]:
!tar -xf release2.3.1.tar.gz    
!tar -xf wi+Blocness_v2.1.bea19.tar.gz

In [3]:
# 압축 풀고
!tar -xf conll2014.tar.gz

## 2.1 CoNLL

Conference on Computational Natural Language Learning (CoNLL)은 매년 자연어처리 관련 기술에 대해 논하는 학회입니다. (학회 설명 보완) 

2013년, 2014년에 더불어 공동 과제 (shared task)를 운영했는데, 에세이내의 문법 오류를 탐지해서 자동으로 고치는 알고리즘을 찾는 과제였습니다. 

해당 데이터는 영어가 두번째 언어인 English as Second Language (ESL) 학생들에 의해 작성된 에세이를 활용했습니다. 50개의 에세이가 존재하며, 2개의 다른 주제에 대한 학생들의 생각을 서술한 글이다. 

- Creator: National University of Singapore
- Year: 2013, 2014
- Purpose: 
    - shared task of the Seventeenth Conference on Computational Natural Language Learning
    - the
goal is to detect and correct the grammatical errors
present in the essay, and return the corrected essay (grammatical error correction)
- Data:
    - Written by ESL students, corrected by Error annotation on the test essays was carried
out by a native speaker of English who is a lecturer
at the NUS Centre for English Language Communication

    - 50 essays (1,312 sentences) on 2 different topics written by 25 South-East Asian undergraduates [Ref. BEA-2019 paper]

- Papers: https://www.aclweb.org/anthology/W13-3601.pdf

In [4]:
from bs4 import BeautifulSoup

In [6]:
# 파일 열고 annotator 1에 의해 라벨링 된 파일
cor = open('conll14st-test-data/noalt/official-2014.1.sgml')

In [7]:
# 파일 읽고
revised = cor.read()

In [12]:
type(revised)

str

In [8]:
# 파싱하고
revisedsoup = BeautifulSoup(revised, 'html.parser')

In [11]:
type(revisedsoup)

bs4.BeautifulSoup

In [7]:
# html tag names
rev_tag_names = set([tag.name for tag in revisedsoup.find_all()])

html tag 별 설명 추가 

In [8]:
rev_tag_names

{'annotation',
 'comment',
 'correction',
 'doc',
 'mistake',
 'p',
 'text',
 'title',
 'type'}

In [22]:
def edit_paragraph(paragraph, start, end, correction):
    return paragraph[:start] + correction + paragraph[end:]

def update_index(paragraph, offset, corrections):
    '''
    paragraph: 원문
    offset: 원본 위치정보
    corrections: 수정해야 하는 단어
    '''
    raw_words = []
    # 1. detect all the words at offset values in the paragraph / O(offset)
    for of in offset:
        raw_words.append(paragraph[of['start_off']:of['end_off']])

    # 2. compare length change between the detected words and corrections (map is faster than list comprehension when it doesn't use lambda) / O(offset)
    adjusted = np.array(list(map(len, corrections))) - np.array(list(map(len, raw_words)))

    # cumulate adjusted values / O(offset)
    for i in range(1, len(adjusted)):
        adjusted[i] += adjusted[i-1]
    
    # 3. adjust the index sequentially / O(offset)
    for i in range(1, len(offset)):
        offset[i]['start_off'] += adjusted[i-1]
        offset[i]['end_off'] += adjusted[i-1]

In [27]:
import numpy as np

In [83]:
all_docs = revisedsoup.find_all('doc')

In [115]:
# strip은 앞뒤 공란 제거
_t = all_docs[0].find('text').text.strip().split('\n\n\n')

In [117]:
_t

['Keeping the Secret of Genetic Testing',
 'What is genetic risk? Genetic risk refers more to your chance of inheriting a disorder or disease. People get certain disease because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you develop heart disease. However for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagonosed out with certain genetic disease, are we suppose to disclose this result to our relatives? My answer is no.',
 "On one hand, we do not want this potential danger causing firghtenning affects in our familie

In [154]:
all_docs = revisedsoup.find_all('doc')
corrected_essays = []

for doc in all_docs:
    
    # extract paragraphs
    paragraph_lists = doc.find('text').text.strip().split('\n\n\n')

    # paragraph 별로 수정 진행
    # make list that contains offset values

    all_mistakes = doc.find_all('mistake')

    for para_idx, para in enumerate(paragraph_lists):
        edits = [mistake.attrs for mistake in all_mistakes if mistake.attrs['start_par'] == f'{para_idx}']

        if len(edits) == 0:
            continue
        
        edits = [{key: int(value) for key, value in edit.items()} for edit in edits]

        # make list that contains correction information 
        corrections = [mistake.find('correction').text for mistake in all_mistakes if mistake.attrs['start_par'] == f'{para_idx}'] #start par과 end par은 항상 같은가?
        
        update_index(para, edits, corrections)

        for edit_idx, edit in enumerate(edits):
            para = edit_paragraph(para, int(edit['start_off']), int(edit['end_off']), corrections[edit_idx])

        paragraph_lists[para_idx] = para

    corrected_essays.append(' '.join(paragraph_lists))

100%|██████████| 50/50 [00:00<00:00, 406.61it/s]


In [155]:
len(corrected_essays)

50

In [157]:
corrected_essays

["Keeping the Secret of Genetic Testing What is genetic risk? Genetic risk refers  to your chance of inheriting a disorder or disease. People get certain diseases because of genetic changes. How much a genetic change tells us about your chance of developing a disorder is not always clear. If your genetic results indicate that you have gene changes associated with an increased risk of heart disease, it does not mean that you definitely will develop heart disease. The opposite is also true. If your genetic results show that you do not have changes associated with an increased risk of heart disease, it is still possible that you will develop heart disease. However, for some rare diseases, people who have certain gene changes are guaranteed to develop the disease. When we are diagnosed with a certain genetic disease, are we supposed to disclose this result to our relatives? My answer is no. On the one hand, we do not want this potential danger to have frightening effects on our families' l

## 2.2 Locness

- Creator: University of Cambridge
- Year: 2019
- Purpose: 
    - 
- Data:
    - represents a wider range of native and learner
English levels and abilities compared to CoNLL
    -  350 essays (4,477 sentences) on approximately 50 topics written by 334 authors from around the world (including native English speakers)

- Papers: https://www.aclweb.org/anthology/W19-4406.pdf

In [ ]:
!wget -O 'conll2014.tar.gz' https://www.comp.nus.edu.sg/~nlp/conll14st/conll14st-test-data.tar.gz

In [ ]:
path = 'wi+locness/json/A.dev.json'

In [ ]:
t = open(path)

In [ ]:
t.read()

'{"text": "It\'s difficult answer at the question \\"what are you going to do in the future?\\" if the only one who has to know it is in two minds. When I was younger I used to say that I wanted to be a teacher, a saleswoman and even a butcher.. I don\'t know why. I would like to study Psychology because one day I would open my own psychology office and help people. It\'s difficult because I\'ll have to study hard and a lot, but I think that if you like a subject, you\'ll study it easier. Maybe I\'ll change my mind, maybe not.", "userid": "21658", "id": "1-352420", "cefr": "A2.ii", "edits": [[0, [[14, 14, " to"], [21, 24, ""], [150, 150, ","], [228, 229, ""], [280, 280, ","], [304, 304, " like to"], [328, 334, "clinic"], [466, 472, "more easily"]]]]}\n{"text": "I think that the public transport will always be in the future. The rich people will buy a car but the poor people always need to use a bus or taxi. I consider that is more convenient to drive a car because you carry on more thi

In [ ]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

## 참고문헌

[1] [The CoNLL-2013 Shared Task on Grammatical Error Correction](https://www.aclweb.org/anthology/W13-3601.pdf)

[2] [The CoNLL-2014 Shared Task on Grammatical Error Correction](https://www.comp.nus.edu.sg/~nlp/conll14st/CoNLLST01.pdf)





In [ ]:
["essay1", "essay2", "essay3"]

In [ ]:
ch6. reference 

In [ ]:
!pip install benepar==0.1.2
!pip install summa
!pip install nltk==3.4.5
!pip install spacy==2.1.0
!python3 -m spacy download en
!pip install scipy

In [ ]:
!pip install tensorflow==1.14.0 


     |████████████████████████████████| 109.3MB 94kB/s 
     |████████████████████████████████| 3.2MB 44.1MB/s 
     |████████████████████████████████| 491kB 48.4MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
import requests
import json
from summa.summarizer import summarize
import benepar
import string
import nltk
from nltk import tokenize
from nltk.tokenize import sent_tokenize
import re
from random import shuffle
import spacy
nlp = spacy.load('en')
#this package is required for the summa summarizer
nltk.download('punkt')
benepar.download('benepar_en2')
benepar_parser = benepar.Parser("benepar_en2")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
sentences[0]

In [ ]:
from string import punctuation

def preprocess(sentences):
    output = []
    for sent in sentences:
        single_quotes_present = len(re.findall(r"['][\w\s.:;,!?\\-]+[']",sent))>0
        double_quotes_present = len(re.findall(r'["][\w\s.:;,!?\\-]+["]',sent))>0
        question_present = "?" in sent
        if single_quotes_present or double_quotes_present or question_present :
            continue
        else:
            output.append(sent.strip(punctuation))
    return output
        
        
def get_candidate_sents(resolved_text, ratio=0.3):
    candidate_sents = summarize(resolved_text, ratio=ratio)
    candidate_sents_list = tokenize.sent_tokenize(candidate_sents)
    candidate_sents_list = [re.split(r'[:;]+',x)[0] for x in candidate_sents_list ]
    # Remove very short sentences less than 30 characters and long sentences greater than 150 characters
    filtered_list_short_sentences = [sent for sent in candidate_sents_list if len(sent)>30 and len(sent)<150]
    return filtered_list_short_sentences

cand_sents = get_candidate_sents(sentences[0])
filter_quotes_and_questions = preprocess(cand_sents)
for each_sentence in filter_quotes_and_questions:
    print (each_sentence)
    print ("\n")

The convenience and high efficiency of using electronic products is being noticed by people worldwide


In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track people




In [ ]:

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final
    

def get_termination_portion(main_string,sub_string):
    combined_sub_string = sub_string.replace(" ","")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ","")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])
                     
    return None
    
def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return get_flattened(last_NP),get_flattened(last_VP)
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


def get_sentence_completions(key_sentences):
    sentence_completion_dict = {}
    for individual_sentence in filter_quotes_and_questions:
        sentence = individual_sentence.rstrip('?:!.,;')
        tree = benepar_parser.parse(sentence)
        last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
        phrases= []
        if last_verbphrase is not None:
            verbphrase_string = get_termination_portion(sentence,last_verbphrase)
            phrases.append(verbphrase_string)
        if last_nounphrase is not None:
            nounphrase_string = get_termination_portion(sentence,last_nounphrase)
            phrases.append(nounphrase_string)

        longest_phrase =  sorted(phrases, key=len,reverse= True)
        if len(longest_phrase) == 2:
            first_sent_len = len(longest_phrase[0].split())
            second_sentence_len = len(longest_phrase[1].split())
            if (first_sent_len - second_sentence_len) > 4:
                del longest_phrase[1]
                
        if len(longest_phrase)>0:
            sentence_completion_dict[sentence]=longest_phrase
    return sentence_completion_dict



sent_completion_dict = get_sentence_completions(filter_quotes_and_questions)

print (sent_completion_dict)


    

{'The convenience and high efficiency of using electronic products is being noticed by people worldwide': ['The convenience and high efficiency of using electronic products is being noticed by', 'The convenience and high efficiency of using electronic products is being'], 'In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track people': ['In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track', 'In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to']}


In [ ]:
sent_completion_dict['The convenience and high efficiency of using electronic products is being noticed by people worldwide']

['The convenience and high efficiency of using electronic products is being noticed by',
 'The convenience and high efficiency of using electronic products is being']

In [ ]:
sent_completion_dict['In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track people']

['In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track',
 'In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to']

In [ ]:

!pip install sentence-transformers==0.2.5.1
!pip install transformers==2.6.0

In [ ]:

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = GPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=tokenizer.eos_token_id)

In [ ]:
from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md
model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:32<00:00, 12.5MB/s]


In [ ]:
import torch

In [ ]:
from nltk import tokenize
import scipy
torch.manual_seed(2020)


def sort_by_similarity(original_sentence,generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.9:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:3]
    

def generate_sentences(partial_sentence,full_sentence):
    input_ids = torch.tensor([tokenizer.encode(partial_sentence)])
    maximum_length = len(partial_sentence.split())+80

    # Actiavte top_k sampling and top_p sampling with only from 90% most likely words
    sample_outputs = model.generate(
        input_ids, 
        do_sample=True, 
        max_length=maximum_length, 
        top_p=0.90, # 0.85 
        top_k=50,   #0.30
        repetition_penalty  = 10.0,
        num_return_sequences=10
    )
    generated_sentences=[]
    for i, sample_output in enumerate(sample_outputs):
        decoded_sentences = tokenizer.decode(sample_output, skip_special_tokens=True)
        decoded_sentences_list = tokenize.sent_tokenize(decoded_sentences)
        generated_sentences.append(decoded_sentences_list[0])
        
    top_3_sentences = sort_by_similarity(full_sentence,generated_sentences)
    
    return top_3_sentences

index = 1
choice_list = ["a)","b)","c)","d)","e)","f)"]
for key_sentence in sent_completion_dict:
    partial_sentences = sent_completion_dict[key_sentence]
    false_sentences =[]
    print_string = "**%s) True Sentence (from the story) :**"%(str(index))
    printmd(print_string)
    print ("  ",key_sentence)
    for partial_sent in partial_sentences:
        false_sents = generate_sentences(partial_sent,key_sentence)
        false_sentences.extend(false_sents)
    printmd("  **False Sentences (GPT-2 Generated)**")
    for ind,false_sent in enumerate(false_sentences):
        print_string_choices = "**%s** %s"%(choice_list[ind],false_sent)
        printmd(print_string_choices)
    index = index+1
    
    print ("\n\n")
        



**1) True Sentence (from the story) :**

   The convenience and high efficiency of using electronic products is being noticed by people worldwide


  **False Sentences (GPT-2 Generated)**

**a)** The convenience and high efficiency of using electronic products is being noticed by more than 20 million consumers.

**b)** The convenience and high efficiency of using electronic products is being noticed by the general public, but this has not been recognized at a rapid pace.

...

**b)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track individuals who do something in the privacy of their own homes instead rather than at work.

**c)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to track every location of someone who has paid the bill and is suspected by law enforcement agencies.

**d)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to keep track of what is going on inside our homes and businesses.

**e)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to identify individuals from the United States and other places then there are good reasons for using it.

**f)** In this essay, I will discuss that if surveillance technology such as RFID (radio-frequency identification) should be used or not to locate terrorist suspects using search warrants and/or law enforcement agencies it is only because of a need for effective monitoring.

In [ ]:
sentences[2]

'\nRisks can be analyzed if there is a record of what was happened in the past. In late nineteenth century, there was a severe air crash happening at Miami international airport. The air cargo of the Valujet plane was on fire after the plane had taken off. Sooner or later, the electric systems were short circuited as a result that the airplane was out of control and nose diving to the ground with extremely high speed. All passengers and pilots were died. After finding and analyzing the black box of the Valujet after the incidence, police investigators found out reasons for causing this severe incidence. Surveillance technology can be used to detect very risky events and prevent its happening in the future, hence to save more lives. In addition, if surveillance technology is even better developed, it can be used to detect the problem before the real accidence has happened.\n'